# __Pringles__, _a documentation_, __part 2__

In this self-documented document about pringles, we will show: 
- model metadata
- a more large-scale model
- working with some python libraries to build the model
- run the simulation doing some parameter sweeping
- use pandas, matplotlib and some pringles's helpers to analyze the results

## What model is on the menu?
**Graphs!** This demo will be all about graphs. Graphs seems like a really nice abstraction that can be applied to several systems / phenoma we might want to simulate. 

In this case, it's no a generic graph, because that will be uninteresting to simulate. Out model is called a **Graph based SIR**, SIR standing for **S**uceptible, **I**nfectious and **R**ecovered. That saying, each node in the graph will be an individual which might start intfected or not, according to a parameter we have in our model called *infection_prob*. When a simulation is initiated, the Node will flip a coin, and with will become infected with $P(startsInfected)=valueOf(infection\_prob)$. Once started, we have to possibilities, depending on how the model initiates:

**TODO**: Add a simple authamata describing a Node behavoiour

Also, it's nice to work with some graph generation and analysis library: networkx. We will work with it, and integrate it with pringles.

Let's start by instantiating the Simulator, and saving the *registry* in an easy access variable.

In [49]:
from pringles.simulator import Simulator, Simulation
from pringles.models import Coupled
from pringles.utils import VirtualTime, new_vtime_aware_axes

In [ ]:
# Instantiate the simulator, targeting our own CD++ distribution (obviously with the atomics we are about to use compiled and registered in it),
#  and the Atomics directory, from which to extact the __Model Metadata__. A bit more on this below.
simulator = Simulator(cdpp_bin_path="user_models/bin", user_models_dir="user_models/")

# Get the Atomic models registry in an easy accessible variable, such as 'r'. Short and concise right?
r = simulator.get_registry()

Now let's look at something interesting. What does targeting the directory where our model sources are located is worth for?

Pringles uses some structured comments we defined in the source files of each model, to instantiate the classes of them for you.

In [12]:
print("  ".join([dict_item for dict_item in dir(r) if not dict_item.startswith("__")]))

Node1  Node10  Node11  Node12  Node13  Node14  Node15  Node16  Node17  Node18  Node19  Node2  Node20  Node3  Node4  Node5  Node6  Node7  Node8  Node9  SUPPORTED_FILE_EXTENSIONS  _add_atomic_class_as_attribute  discover_atomics  discovered_atomics  user_models_dir


Those NodeN, with N a number, are our node models. Their classes and ports have been already loaded into the pringles registry.

Below, you'll find a detailed explanation of model metadata.

In [36]:
r.Node5("test").get_port("in4")

## Model Metadata

Let's suppose that you don't know abot this model metadata thingy, and that you only have one atomic model in the simulator called __Queue__.
In that case, for it to be accessible from pringles, you would have to define a class, inheriting from Atomic. Furthermore, you would need to declare by hand each port the model has.

Let's see the __Queue__ example, assuming it just hast two input ports, __job__ and __ready___; and one output port, __jobs_status__ .

```python
from pringles.models import Atomic

class Queue(Atomic):
    def __init__(self, name: str, **model_params: str):
        super().__init__(name, **model_params)
        self.add_output_port("job_status")
        self.add_input_port("job")
        self.add_input_port("ready")
```

In my opinions, it's annoying having to write the whole constructor over and over for each Atomic one has already written in C++. Wouldn't it be nice if __pringles__ could read somehow the C++ model implementation?

That's when __model metadata__ comes handy!

Model metadata is a comment format we defined for you to write over each header file of each Atomic you want for pringles to discover. Let's say (following the prior example) you want the _queue_ to be discovered. In that case, you'd just go to _queue.h_, and somewhere on the file (preferrably over the class definition, just for setting a convention) and add a comment like this one:

```c++
/*
@ModelMetadata
name:   Queue
input_ports: job, ready
output_ports: job_status
*/

class Queue : public Atomic {
    /*
    All you lovely CD++ queue impl.
    */
}
```

Let's analyze the syntax a little more, just to be clear:

```c++
/*
@ModelMetadata
name: Queue                     <- This is the atomic model name, which will become in python the class name you'll instance.
input_ports: job, ready         <- comma-separated-list of input ports names
output_ports: job_status        <- comma-separated-list of output ports names
*/
```

## Building the model

Now, back to our model. We are going to work with atomics named **NodeN**, with $1 <= n <= 20$, where NodeN is a vertex with N different input, and output ports. In other words, the *Node5* atomic model has 5 input ports named *in0* to *in4*, and the same for *outM*.

According to the model description above, this changes the way in which the *infection* part of the node's behaviour work. Notice that it's not related to the degree-graph-sense of the node, since by using a Node1 (just one input and output port), we can make a vertex of degree 50.

We will describe to approaches for building the network (graph) of individuals we want to simulate:
- A pure pringles approach
- A NetworkX - Pringles mixup

### Pure pringles

For this approach, we will build a fully-connected-graph. For this, we will use just pringles, and leverage the way in which we built our Node atomics. 

First, we will a build a function for building a $N$ nodes fully-connected top model, also propagating the parameters of both the $P(infection)$, *infection_prob*, and the inter-infections exponential time mean, *exp_lambda*.

Then, we will instantiate a sample model (the parameters should be nonsense), and run a simple simulation.

In [38]:
def build_complete_graph_of_nodes(n, exp_lambda, infection_rate):
    # Instantiate 'n' Node1 atomics. Remember that his ones just have one input port (in0), and one ouput (out0)
    nodes = [r.Node1("node%d" % (i), exp_lambda=exp_lambda, infection_prob=infection_rate) for i in range(n)]
    # This will be our top coupled, called 'top' obviously, and will all the nodes we created above
    graph = Coupled("top", nodes)
    # For each pair of different nodes we created, we will connect their only ports.
    for i in range(0,n):
        for j in range(0,n):
            if i != j:
                graph.add_coupling(nodes[i].get_port("out0"), nodes[j].get_port("in0"))
    return graph

In [50]:
# Now, let's instantiate a sample 5 vertexes SIR model.
sample_five_node_graph = build_complete_graph_of_nodes(5, 3, .25)
# NOTE-INTERNAL: I have to run the cell twice in order for Diagrammer for rendering the HTML-representation. Also, sometimes is rendered horribly.
# Notice that we binded Jupyter's rich display environment with our Model implementation.
sample_five_node_graph

In [51]:
# We instantiate the Simulation we are about to run. In case you didn't notice, Simulation objects are immutable once instantiated. 
# This helps having one container for the whole simulation you've ran, and will make things be more ordered.

# The simulation will be of our recently-created 5-node top model, and will be ran for 60 seconds. You can check out the other VirtualTime helpers, or use the main constructor.
sample_simulation = Simulation(sample_five_node_graph, VirtualTime.of_seconds(60))
results = simulator.run_simulation(sample_simulation)

In [52]:
print(results.logs_dfs.keys())

dict_keys(['node4', 'node3', 'node2', 'node1', 'node0', 'top', 'ParallelRoot'])


In [ ]:
import pandas as pd

# Concatenating all nodes external transitions
nodes_external_transitions = pd.concat([results.logs_dfs[node_id] for node_id in results.logs_dfs.keys() if node_id.startswith("node")])
nodes_external_transitions = nodes_external_transitions.sort_values(by=["time"])
nodes_external_transitions

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
infections_count = nodes_external_transitions.groupby("time").count()
infections_count = infections_count.drop([0, 1, "message_type", "model_origin", "port", "model_dest"], axis=1)
infections_count = infections_count.rename({"value":"infections_count"}, axis=1)

In [ ]:
accum_infections = [0]*len(list(infections_count.infections_count))
for i, c in enumerate(list(infections_count.infections_count)):
    if i == 0:
        accum_infections[i] = c
    else:
        accum_infections[i] = accum_infections[i-1] + c

In [ ]:
from matplotlib.ticker import FuncFormatter
ax = new_vtime_aware_axes()
ax.set_xlim(0, max([float(vtime) for vtime in vtime_indexes])+float(VirtualTime.from_number(1000)))
ax.plot(vtime_indexes, list(accum_infections))

### Configuration model graph generation

In [ ]:
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
import random

In [ ]:
# Generating configuration model degree sequence
alpha = 10
beta = 1
node_count = 75
degree_sequence = [int(random.gammavariate(alpha, beta)) for i in range(node_count)]
if sum(degree_sequence) % 2 == 1:
    degree_sequence[-1] += 1

In [ ]:
import time
import math
random_seed = int(time.time()*1000) << 2

In [ ]:
steffi_graph = nx.configuration_model(degree_sequence, seed=random_seed)

In [ ]:
# Drawing graph
pos = graphviz_layout(steffi_graph, root=0)
nx.draw(steffi_graph, pos, with_labels=False, alpha=.3, node_size=20)
plt.show()

In [ ]:
from typing import Dict
def build_graph_model_from_nx_graph(graph: nx.Graph, exp_lambda, infection_rate):
    nodes_degrees = [graph.degree(i) for i in range(len(graph.nodes))]
    max_degree = max(nodes_degrees)
    if max_degree > 20:
        raise Exception("Do no have models of degree above 20")
    nodes = [getattr(r,"Node%d" % (degree))("node%d" % (i), exp_lambda=exp_lambda, infection_prob=infection_rate) 
             for i, degree in enumerate(nodes_degrees)]
    graph = Coupled("top", nodes)
    
    _last_out_node = {}
    _last_in_node = {}

    def do_get_last_port(u: int, dic: Dict[int, int], portname: str):
        if u not in dic:
            dic[u] = 0
            return portname + "0"
        else:
            last = dic[u]
            dic[u] = last+1
            return portname + "%d" % (last+1)

    def last_out_node(u):
        return do_get_last_port(u, _last_out_node, "out")
    def last_in_node(u):
        return do_get_last_port(u, _last_in_node, "in")
    
    # Connecting nodes according to graph
    for u,v in steffi_graph.edges():
        graph.add_coupling(nodes[u].get_port(last_out_node(u)), nodes[v].get_port(last_in_node(v)))
    return graph

In [ ]:
nx_graph_coupled = build_graph_model_from_nx_graph(steffi_graph, 2, .5)

In [ ]:
results = simulator.run_simulation(nx_graph_coupled, VirtualTime.of_minutes(10))

In [ ]:
results